# Ingesting the data into the VectorDB

This notebook is just to get things started - don't want to use the Api latency when I can do it without it

In [6]:
import numpy as np
import pandas as pd
from pymilvus import (
    connections,
    utility,
    FieldSchema, 
    CollectionSchema, 
    DataType,
    Collection,
)
from sentence_transformers import SentenceTransformer 